In [1]:
%cd C:\Users\ehate\Desktop\ASP\ASP_code_projects\ASP_predictor
from src.utils.cross_validation import CrossValidation
from src.models.random_forest_regression import RandomForestRegression

import pandas as pd
from pathlib import Path

C:\Users\ehate\Desktop\ASP\ASP_code_projects\ASP_predictor


In [2]:
# param_grid = {
#     "criterion": ['squared_error'],
#     "n_estimators": [30],
#     "max_depth": [None, 10, 20, 30, 40, 50],
#     "min_samples_split":[2,5,10,15,20],
#     "min_samples_leaf": [1,5,10,15,20],
#     "max_features": ["sqrt", "log2", 0.3, 0.5, 1.0],
#     "max_samples": [None, 0.5, 0.7, 0.9],
#     "min_weight_fraction_leaf": [0.0],
#     "bootstrap": [True]
# }

param_grid = {
    "n_estimators": [30],
    "max_depth": [None, 10],
    "min_samples_split":[2,5],
}

train_data = pd.read_parquet(Path("data/intermediate/datasets/ASP_train.parquet"))
val_data = pd.read_parquet(Path("data/intermediate/datasets/ASP_val.parquet"))

model_configs = dict(         
                 input_features = ['6.9H', '6.9V', '10.7H', '10.7V', '18H','18V', '23H', '23V', '36H', '36V', '89H', '89V', 't2m','tcwv', 'tclw', 'tciw'],
                 target_feature = 'SD_mean', 
                 weight_feature = 'num_samples',
                 oob_score = False, 
                 random_state = None
                )

CV = CrossValidation(RandomForestRegression, model_configs=model_configs, cv_folds=2, params=param_grid)
out=CV.perform_grid_search(train_data, val_data, nproc=3)

100%|██████████| 8/8 [01:27<00:00, 10.89s/it]


In [26]:
out.performance_cube.mse

<xarray.DataArray 'mse' (cv_fold: 2, n_estimators: 1, max_depth: 2,
                         min_samples_split: 2)> Size: 64B
array([[[[0.00339131, 0.00336863],
         [0.00346279, 0.00335783]]],


       [[[0.00356022, 0.00350251],
         [0.00356671, 0.00352022]]]])
Coordinates:
  * cv_fold            (cv_fold) int64 16B 0 1
  * n_estimators       (n_estimators) object 8B '30'
  * max_depth          (max_depth) object 16B '10' 'None'
  * min_samples_split  (min_samples_split) object 16B '2' '5'

In [ ]:
def reverse_xarray_selection_to_params(x):
    """
    Given e.g. x = ds.mse.sel(...)
    return the corresponding Python param dict, restoring types:
      "None" → None
      "10"   → int(10)
      "3.14" → float(3.14)
      everything else stays as string.
    """
    # If user passed Dataset/DataArray — ensure 0-D DataArray

    params = {}
    for dim, raw in x.coords.items():
        v = raw.item()  # Python scalar

        # --- unstringify back to meaningful Python literal ---
        if v == "None":
            params[dim] = None
        else:
            try:
                params[dim] = int(v)
            except (ValueError, TypeError):
                try:
                    params[dim] = float(v)
                except (ValueError, TypeError):
                    params[dim] = v  # leave as string

    return params, float(x.values.squeeze())

In [40]:
out.performance_cube.mse.where(out.performance_cube.mse==out.performance_cube.mse.min(), drop=True)

<xarray.DataArray 'mse' (cv_fold: 1, n_estimators: 1, max_depth: 1,
                         min_samples_split: 1)> Size: 8B
array([[[[0.00335783]]]])
Coordinates:
  * cv_fold            (cv_fold) int64 8B 0
  * n_estimators       (n_estimators) object 8B '30'
  * max_depth          (max_depth) object 8B 'None'
  * min_samples_split  (min_samples_split) object 8B '5'

In [46]:
best_idx

<xarray.DataArray 'mse' (n_estimators: 1, max_depth: 1, min_samples_split: 1)> Size: 8B
array([[[0.00343557]]])
Coordinates:
  * n_estimators       (n_estimators) object 8B '30'
  * max_depth          (max_depth) object 8B '10'
  * min_samples_split  (min_samples_split) object 8B '5'

In [50]:
mean = out.performance_cube.mse.mean('cv_fold')
best_idx = mean.where(mean == mean.min(), drop=True)
best_params, best_mse = reverse_xarray_selection_to_params(best_idx)
print("Best params:", best_params)
print("Best MSE:", best_mse)

Best params: {'n_estimators': 30, 'max_depth': 10, 'min_samples_split': 5}
Best MSE: [[[0.00343557]]]


In [56]:
float(best_mse.squeeze())

0.0034355653127390875

In [37]:
x = out.performance_cube.mse.sel(n_estimators='30', max_depth='None', min_samples_split='2', cv_fold=1)
xo = reverse_xarray_selection_to_params(x, var="mse")

In [38]:
xo

({'cv_fold': 1, 'n_estimators': 30, 'max_depth': None, 'min_samples_split': 2},
 0.003566712231382232)

In [27]:
best = out.performance_cube.mse.argmin().values


In [28]:
best

array(3)

In [25]:
out.performance_cube.mse.min()

<xarray.DataArray 'mse' ()> Size: 8B
array(0.00335783)

In [29]:
df = out.performance_cube.to_dataframe().reset_index()
df.loc[best]

cv_fold                     0
n_estimators               30
max_depth                None
min_samples_split           5
mse                  0.003358
Name: 3, dtype: object

In [13]:
out.performance_cube.mse.argmin()

<xarray.DataArray 'mse' ()> Size: 8B
array(3)